In [1]:
import os 
import cv2
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, cross_val_predict, \
                                    GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,\
                             AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.externals import joblib

import pprint


In [2]:
ls

dqn_LunarLander-v2_weights.h5f     README.txt
lunar_lander_data_gen.py*          rl/
lunar_lander_dl_player.py*         Task1.ipynb
lunar_lander_ml_images_player.py*  Task2.ipynb
lunar_lander_ml_states_player.py*  Task3.py*


### Reading data, specifying parameters for experiment, sampling data

In [3]:
#Plug in data file

#Other User
#data_file = "state_vectors_....etc.csv"

#Andy
# data_file = "state_vectors_2018-04-21-20-41-25.csv"

#Conor
data_file = "../state_vectors_2018-04-20-15-12-38.csv"

sample_nr_rows = 6000
grid_search_cv_folds = 5
cv_experiment_folds = 10

In [4]:
# load dataset
df = pd.read_csv(data_file)

# take sample of required size
sample_df = df.sample(sample_nr_rows)

In [5]:
# split data into X,y (descriptive features, target feature)
X,y = sample_df[[col for col in list(sample_df) if col!='action']], sample_df['action']

### Choosing algorithms, grid searching to find a set of sensible parameters

In [6]:
# choose 3 classifiers to go in here
clfs = [RandomForestClassifier, MLPClassifier, BaggingClassifier]

In [7]:
# dictionary for grid search parameters
param_grids= {
    'RandomForestClassifier': {
        'criterion' : ['gini', 'entropy'],
        'n_estimators' : [5,10,15],
        'max_features' : [None, 5, 8]
    },
    'MLPClassifier': {
        'activation' : ['relu', 'logistic'],
        'hidden_layer_sizes' : [(100,), (200,), (300,)],
        'solver' : ['adam', 'lbfgs'],
        'learning_rate' : ['adaptive', 'constant']
    },
    'BaggingClassifier' : {
        'n_estimators' : [5,10,15],
        'max_samples' : [0.5,0.75,1],
        'max_features' : [0.5,0.75,1]
    }
}

In [9]:
# dictionary to store the best set of parameters and the best score for each clf
tuned_clfs={}
stored_grid_searches={}
# running grid search for each clf
for clf in clfs:
    clf_name = str(clf).split(".")[-1][:-2]
    current_GS = GridSearchCV(clf(), 
                              param_grids[clf_name],
                              cv=grid_search_cv_folds,
                              verbose=0, 
                              return_train_score=True).fit(X,y)
    
    #display(pd.DataFrame(current_GS.cv_results_)
    stored_grid_searches[clf_name]=current_GS # store grid search object for each classifier
    
    # store best params and best score and search results in a sub dictionary for clf

### Inspecting grid search objects to select set of parameters

In [59]:
selected_params={} # dict to store selcted parameters for each classifier

In [60]:
RF_GS_df = pd.DataFrame(stored_grid_searches['RandomForestClassifier'].cv_results_).sort_values('mean_test_score',ascending=False)
RF_GS_df.head(3)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_criterion,param_max_features,param_n_estimators,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
2,0.356062,0.004777,0.836333,0.995708,gini,None,15,"{'criterion': 'gini', 'n_estimators': 15, 'max...",1,0.834443,...,0.833333,0.996250,0.845705,0.995626,0.846411,0.995627,0.006865,0.000095,0.009085,0.000283
8,0.317243,0.004675,0.831333,0.994875,gini,8,15,"{'criterion': 'gini', 'n_estimators': 15, 'max...",2,0.828619,...,0.834167,0.995833,0.830692,0.993751,0.838898,0.995627,0.006156,0.000062,0.004945,0.000764
7,0.211386,0.003456,0.828667,0.989958,gini,8,10,"{'criterion': 'gini', 'n_estimators': 10, 'max...",3,0.836106,...,0.825000,0.991250,0.815680,0.990210,0.828881,0.989588,0.003445,0.000104,0.007974,0.000762


In [61]:
print("best params:", stored_grid_searches['RandomForestClassifier'].best_params_)
print("best score:", stored_grid_searches['RandomForestClassifier'].best_score_)
selected_params['RandomForestClassifier'] = stored_grid_searches['RandomForestClassifier'].best_params_

best params: {'criterion': 'gini', 'n_estimators': 15, 'max_features': None}
best score: 0.836333333333


In [62]:
MLP_GS_df = pd.DataFrame(stored_grid_searches['MLPClassifier'].cv_results_).sort_values('mean_test_score',ascending=False)
MLP_GS_df.head(3)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_activation,param_hidden_layer_sizes,param_learning_rate,param_solver,params,rank_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
8,0.593428,0.004500,0.754167,0.754793,relu,"(300,)",adaptive,adam,"{'learning_rate': 'adaptive', 'activation': 'r...",1,...,0.757500,0.762708,0.747289,0.752552,0.744574,0.748230,0.134222,0.000602,0.006875,0.005937
6,0.526278,0.002162,0.746667,0.748169,relu,"(200,)",constant,adam,"{'learning_rate': 'constant', 'activation': 'r...",2,...,0.750833,0.758333,0.739783,0.746303,0.735392,0.733444,0.085068,0.000305,0.007617,0.009242
16,2.561351,0.007281,0.746500,0.748377,logistic,"(200,)",adaptive,adam,"{'learning_rate': 'adaptive', 'activation': 'l...",3,...,0.746667,0.750833,0.729775,0.740262,0.737896,0.738442,1.118944,0.000680,0.011571,0.008787


In [63]:
print("best params:", stored_grid_searches['MLPClassifier'].best_params_)
print("best score:", stored_grid_searches['MLPClassifier'].best_score_)
selected_params['MLPClassifier'] = stored_grid_searches['MLPClassifier'].best_params_

best params: {'learning_rate': 'adaptive', 'activation': 'relu', 'hidden_layer_sizes': (300,), 'solver': 'adam'}
best score: 0.754166666667


In [64]:
BAG_GS_df = pd.DataFrame(stored_grid_searches['BaggingClassifier'].cv_results_).sort_values('mean_test_score',ascending=False)
BAG_GS_df.head(3)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_features,param_max_samples,param_n_estimators,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
14,0.219637,0.005207,0.811833,0.983000,0.75,0.75,15,"{'max_samples': 0.75, 'n_estimators': 15, 'max...",1,0.831115,...,0.813333,0.983958,0.814012,0.982504,0.813022,0.984173,0.004596,0.000454,0.013891,0.000995
11,0.199638,0.005623,0.800167,0.952376,0.75,0.5,15,"{'max_samples': 0.5, 'n_estimators': 15, 'max_...",2,0.800333,...,0.795833,0.951042,0.795663,0.951260,0.808848,0.947522,0.020241,0.000630,0.004781,0.003275
10,0.132031,0.004293,0.789667,0.941584,0.75,0.5,10,"{'max_samples': 0.5, 'n_estimators': 10, 'max_...",3,0.813644,...,0.794167,0.942500,0.796497,0.941887,0.778798,0.935235,0.018196,0.001088,0.016492,0.003364


In [65]:
print("best params:", stored_grid_searches['BaggingClassifier'].best_params_)
print("best score:", stored_grid_searches['BaggingClassifier'].best_score_)
selected_params['BaggingClassifier'] = stored_grid_searches['BaggingClassifier'].best_params_

best params: {'max_samples': 0.75, 'n_estimators': 15, 'max_features': 0.75}
best score: 0.811833333333


In [68]:
# cross validation experiment for clfs using hyper-params from above
cv_scores = {}
for clf in clfs:
    clf_name = str(clf).split(".")[-1][:-2]
    params = selected_params[clf_name]
    cv_scores[clf_name] = np.mean(cross_val_score(clf(**params), X, y, cv=cv_experiment_folds)) 


In [69]:
# print the avg cross validation scores for the clfs using their best params
pprint.pprint(cv_scores)

{'BaggingClassifier': 0.80549807909758009,
 'MLPClassifier': 0.73348540640859772,
 'RandomForestClassifier': 0.84049513139104304}


In [12]:
#"Save something, i'm giving up on you."
#joblib.dump(clf, 'states_learner.pkl')

### Investigating how these algorithms perform with more data, using full data set from input file

In [70]:
# split data into X,y (descriptive features, target feature)
X_big,y_big = df[[col for col in list(df) if col!='action']], df['action']

In [74]:
full_df_cv_scores={}
for clf in clfs:
    clf_name = str(clf).split(".")[-1][:-2]
    params = selected_params[clf_name]
    full_df_cv_scores[clf_name] = np.mean(cross_val_score(clf(**params), X_big, y_big, cv=cv_experiment_folds)) 

In [75]:
pprint.pprint(full_df_cv_scores)

{'BaggingClassifier': 0.8625343104640848,
 'MLPClassifier': 0.91909871823683942,
 'RandomForestClassifier': 0.90721220400154512}
